# KerasNLP ile Transformer'ı sıfırdan eğitmek.

KerasNLP, son teknoloji metin işleme modelleri oluşturmayı kolaylaştırmayı amaçlamaktadır. Bu notebook ile transformer'ları sıfırdan nasıl basit bir şekilde eğiteceğimizi göstereceğiz.

Bu kılavuz üç bölüme ayrılmıştır:

1. Kurulum
2. Ön eğitimli bir Transformer Modeli
3. Sınıflandırma görevimizde Transformer modelinde fine-tuning işlemi.

## Kurulum

Paketlerimizi içeri aktaralım: `keras_nlp`, `keras` ve `tensorflow`.

[buraya bir göz atmalısın](https://keras.io/api/mixed_precision/), kısaca eğitimi hızlandırmak için hesaplamalarımızın çoğunu 16 bit (32 bit yerine) float sayılarla çalıştırıyoruz.
Bir Transformer'ı eğitmek biraz zaman alabilir, bu yüzden daha hızlı eğitim için tüm engelleri kaldırmak önemlidir.

In [1]:
!pip install -q --upgrade keras-nlp tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.9 MB/s eta 0:00:00


In [2]:
import os

import keras_nlp
import tensorflow as tf
from tensorflow import keras

policy = keras.mixed_precision.Policy("mixed_float16")
keras.mixed_precision.set_global_policy(policy)

Sonra dataset'leri indirebiliriz.

- [SST-2](https://paperswithcode.com/sota/sentiment-analysis-on-sst-2-binary)   Bir metin
sınıflandırma veri seti ve "nihai hedefimiz" var. Bu veri kümesi genellikle dil modellerinde kıyaslama yapmak için kullanılır.

- [WikiText-103](https://paperswithcode.com/dataset/wikitext-103): Ön eğitim için kullanılan İngilizce wikipedia'dan öne çıkan makaleler koleksiyonu veri seti.

Son olarak, daha sonra alt kelime belirteci yapmak için bir WordPiece sözlüğü indireceğiz.


In [3]:
# Ön eğitim verilerini indirin.
keras.utils.get_file(
    origin="https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip",
    extract=True,
)
wiki_dir = os.path.expanduser("~/.keras/datasets/wikitext-103-raw/")

# İnce ayar verilerini indirin.
keras.utils.get_file(
    origin="https://dl.fbaipublicfiles.com/glue/data/SST-2.zip",
    extract=True,
)
sst_dir = os.path.expanduser("~/.keras/datasets/SST-2/")

# Sözlük verilerini indirin.
vocab_file = keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/keras-nlp/examples/bert/bert_vocab_uncased.txt",
)

231508/231508 [==============================] - 0s 1us/step


Ardından, eğitim sırasında kullanacağımız bazı hiperparametreleri tanımlıyoruz.

In [4]:
# Ön işleme parametreleri.
PRETRAINING_BATCH_SIZE = 128
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32

# Model parametreleri.
NUM_LAYERS = 3
MODEL_DIM = 256
INTERMEDIATE_DIM = 512
NUM_HEADS = 4
DROPOUT = 0.1
NORM_EPSILON = 1e-5

# Eğitim parametreleri.
PRETRAINING_LEARNING_RATE = 5e-4
PRETRAINING_EPOCHS = 8
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3

### Veri yükle
Verilerimizi [tf.data](https://www.tensorflow.org/guide/data) ile yüklüyoruz, tf.data bize metni simgeleştirme ve ön işleme için giriş işlem hatlarını tanımlamamızı sağlar.

In [ ]:
# SST-2 yüklenmesi.
sst_train_ds = tf.data.experimental.CsvDataset(
    sst_dir + "train.tsv", [tf.string, tf.int32], header=True, field_delim="\t"
).batch(FINETUNING_BATCH_SIZE)
sst_val_ds = tf.data.experimental.CsvDataset(
    sst_dir + "dev.tsv", [tf.string, tf.int32], header=True, field_delim="\t"
).batch(FINETUNING_BATCH_SIZE)

# wikitext-103'ü yükleyin ve kısa satırları filtreleyin.
wiki_train_ds = (
    tf.data.TextLineDataset(wiki_dir + "wiki.train.raw")
    .filter(lambda x: tf.strings.length(x) > 100)
    .batch(PRETRAINING_BATCH_SIZE)
)
wiki_val_ds = (
    tf.data.TextLineDataset(wiki_dir + "wiki.valid.raw")
    .filter(lambda x: tf.strings.length(x) > 100)
    .batch(PRETRAINING_BATCH_SIZE)
)


print(sst_train_ds.unbatch().batch(4).take(1).get_single_element())

`SST-2` veri setimizin nispeten kısa film incelemesi parçacıkları içerdiğini görebilirsiniz.
Amacımız, snippet'in duyarlılığını tahmin etmektir. 1 etiketi pozitif duygu ve 0 negatif duygu etiketini temsil eder. Bu dataset için görevimiz bu etiketleri tahmin etmektir.

### Bir temel oluşturmak

Pozitif veya negatif bir ağırlık öğrendiğimiz basit bir kelime çantası modeli eğiteceğiz.


In [6]:
# Bu katman, giriş cümlemizi aynı boyutta 1'ler ve 0'lardan oluşan bir listeye dönüştürecek.

multi_hot_layer = keras.layers.TextVectorization(
    max_tokens=4000, output_mode="multi_hot"
)
multi_hot_layer.adapt(sst_train_ds.map(lambda x, y: x))
regression_layer = keras.layers.Dense(1, activation="sigmoid")
inputs = keras.Input(shape=(), dtype="string")
outputs = regression_layer(multi_hot_layer(inputs))
baseline_model = keras.Model(inputs, outputs)
baseline_model.compile(loss="binary_crossentropy", metrics=["accuracy"])
baseline_model.fit(sst_train_ds, validation_data=sst_val_ds, epochs=5)

Epoch 1/5
2105/2105 [==============================] - 16s 5ms/step - loss: 0.6131 - accuracy: 0.6873 - val_loss: 0.5367 - val_accuracy: 0.7569
Epoch 2/5
2105/2105 [==============================] - 11s 5ms/step - loss: 0.5250 - accuracy: 0.7603 - val_loss: 0.4888 - val_accuracy: 0.7821
Epoch 3/5
2105/2105 [==============================] - 12s 6ms/step - loss: 0.4787 - accuracy: 0.7869 - val_loss: 0.4682 - val_accuracy: 0.7913
Epoch 4/5
2105/2105 [==============================] - 12s 6ms/step - loss: 0.4486 - accuracy: 0.8021 - val_loss: 0.4591 - val_accuracy: 0.7982
Epoch 5/5
2105/2105 [==============================] - 12s 6ms/step - loss: 0.4273 - accuracy: 0.8117 - val_loss: 0.4561 - val_accuracy: 0.7982


## Ön Eğitim

Bir `transformer` eğiteceğiz, 
girdimizdeki her kelimeyi düşük boyutlu bir vektör olarak gömeceğiz.Maskeli Dil adlı denetimsiz bir eğitim hedefi kullanacağız.(MaskedLM).

Esasen, büyük bir "eksik kelimeyi tahmin et" oyunu oynayacağız. Her giriş için
örnekte, girdi verilerimizin %25'ini gizleyeceğiz ve modelimizi, istediğimiz parçaları tahmin edecek şekilde eğiteceğiz.

### MaskedLM görevi için verileri ön işleme

MaskedLM görevi için metin ön işlememiz iki aşamada gerçekleşecektir.

1. Girdi metnini belirteç kimliklerinin tamsayı dizilerine dönüştürün.
2. Tahmin etmek için girdimizdeki belirli konumları maskeleyin.

`keras_nlp.tokenizers.Tokenizer` kullanarak
metni tamsayı belirteç kimlikleri dizilerine dönüştürebiriz.

Özellikle, `keras_nlp.tokenizers.WordPieceTokenizer`'ı kullanacağız.
*sub-word*  tokenizasyonu, modelleri büyük ölçekte eğitirken popülerdir. Temel olarak, modelimizin yaygın olmayan sözcüklerden öğrenmesine izin verirken,
eğitim setimizdeki her kelime için devasa bir kelime dağarcığı gerektiriyor.

Yapmamız gereken ikinci şey, MaskedLM görevi için girdimizi maskelemek. Bunu yapmak için kullanabiliriz
`keras_nlp.layers.MaskedLMMaskGenerator`, her birinde rastgele bir jeton seti seçecektir.


Tokenizer ve maskeleme katmanının her ikisi de bir çağrı içinde kullanılabilir.
[tf.data.Dataset.map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map).
GPU'muz veya TPU'muz çalışırken CPU'daki her partiyi verimli bir şekilde önceden hesaplamak için `tf.data`'yı kullanabiliriz.


In [ ]:
# sequence_length ayarı, token çıktılarını şekillendirecek şekilde kırpacak veya dolduracaktır. pad sequence vs. vs.
# (batch_size, SEQ_LENGTH).
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab_file,
    sequence_length=SEQ_LENGTH,
    lowercase=True,
    strip_accents=True,
)
# mask_selection_length ayarı, maske çıktılarını şekle göre kırpar veya doldurur
# (batch_size, PREDICTIONS_PER_SEQ).
masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=MASK_RATE,
    mask_selection_length=PREDICTIONS_PER_SEQ,
    mask_token_id=tokenizer.token_to_id("[MASK]"),
)


def preprocess(inputs):
    inputs = tokenizer(inputs)
    outputs = masker(inputs)
    # Maskeleme katmanı çıktılarını bir (features, labels, ve weights) olarak ayırın
    # tuple that we can use with keras.Model.fit().
    features = {
        "token_ids": outputs["token_ids"],
        "mask_positions": outputs["mask_positions"],
    }
    labels = outputs["mask_ids"]
    weights = outputs["mask_weights"]
    return features, labels, weights


# Önceden işlenmiş partileri CPU üzerinde hareket halindeyken önceden hesaplamak için prefetch() kullanıyoruz.
pretrain_ds = wiki_train_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
pretrain_val_ds = wiki_val_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Hücreyi her çalıştırdığınızda maskeler değişecektir.
print(pretrain_val_ds.take(1).get_single_element())

Yukarıdaki blok, veri kümemizi bir `(features, labels, weights)` demetine ayırır ve bunu
doğrudan `keras.Model.fit()` öğesine iletir.

İki özelliğimiz var:

1. `"token_ids"`, bazı tokenlerin maske token kimliğimizle değiştirildiği yer.
2. `"mask_positions"`,hangi tokenleri maskelediğimizi takip eder.

Etiketlerimiz, basitçe maskelediğimiz kimliklerdir.

Tüm diziler aynı sayıda maskeye sahip olmayacağından, aynı zamanda
"sample_weight" tensörü, dolgulu etiketleri kayıp fonksiyonumuzdan kaldırır.

### Transformer encoder oluşturalım.

KerasNLP, hızlı bir şekilde bir Transformer encoder oluşturmak için tüm yapı taşlarını sağlar.

Giriş token ids'leri ilk önce gömmek için `keras_nlp.layers.TokenAndPositionEmbedding` kullanıyoruz.
Bu katman aynı anda iki embeddingi öğrenir -- biri cümledeki kelimeler için, diğeri bir cümledeki tamsayı konumları için. Çıktı gömme basitçe ikisinin toplamıdır.

Sonra bir dizi `keras_nlp.layers.TransformerEncoder` katmanı ekleyebiliriz. Bunlar bir dikkat mekanizması (ardından çok katmanlı bir algılayıcı bloğu) kullanarak Transformer modelinin başarımını artırmak içindir.

Bu modelin çıktısı, giriş token id'si başına kodlanmış bir vektör olacaktır.

In [8]:
inputs = keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32)

# Tokenleri konumsal bir embedding ile gömün.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=tokenizer.vocabulary_size(),
    sequence_length=SEQ_LENGTH,
    embedding_dim=MODEL_DIM,
)
outputs = embedding_layer(inputs)

# Embedding'imize dropout ve normalization uygulayalım
outputs = keras.layers.LayerNormalization(epsilon=NORM_EPSILON)(outputs)
outputs = keras.layers.Dropout(rate=DROPOUT)(outputs)

# Bir dizi encoder bloğu ekleyin
for i in range(NUM_LAYERS):
    outputs = keras_nlp.layers.TransformerEncoder(
        intermediate_dim=INTERMEDIATE_DIM,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        layer_norm_epsilon=NORM_EPSILON,
    )(outputs)

encoder_model = keras.Model(inputs, outputs)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128)]             0         
                                                                 
 token_and_position_embeddin  (None, 128, 256)         7846400   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 layer_normalization (LayerN  (None, 128, 256)         512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 256)          0         
                                                                 
 transformer_encoder (Transf  (None, 128, 256)         527104    
 ormerEncoder)                                             

### Transformer'ı önceden eğitin

`keras_nlp.layers.MaskedLMHead` kullanarak encoder bloğumuzu MaskedLM görevini yapacak şekilde eğitebiliriz.

Bu katman, bir girdi olarak token kodlamalarını ve bir başkası olarak da bizim belirlediğimiz konumları alacaktır.
orijinal girişte maskelenmiş. Maskelediğimiz token kodlamalarını toplayacak ve
onları tüm kelime dağarcığımız üzerindeki tahminlere geri dönüştürün.

Bununla, ön eğitimi derlemeye ve çalıştırmaya hazırız. Bunu bir
colab kullanarak, yaklaşık bir saat süreceğini unutmayın. Training Transformer, yoğun hesaplamasıyla ünlüdür. Dolayısıyla bu nispeten küçük Transformatör bile biraz zaman alacaktır.

In [ ]:
# Maskelenmiş bir dil modeli başlığı ekleyerek eğitim öncesi modeli oluşturun.
inputs = {
    "token_ids": keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32),
    "mask_positions": keras.Input(shape=(PREDICTIONS_PER_SEQ,), dtype=tf.int32),
}

# Encode tokens.
encoded_tokens = encoder_model(inputs["token_ids"])

# Her maskelenmiş giriş tokeni için bir çıkış sözcüğü tahmin edin.
# Encoding vektörlerimizden kelime logitlerine yansıtmak için giriş tokenlerini yerleştirmeyi kullanıyoruz, bunun eğitim verimliliğini artırdığı gösterilmiştir.
outputs = keras_nlp.layers.MaskedLMHead(
    embedding_weights=embedding_layer.token_embedding.embeddings,
    activation="softmax",
)(encoded_tokens, mask_positions=inputs["mask_positions"])

# Ön eğitim modelimizi tanımlayın ve derleyin.
pretraining_model = keras.Model(inputs, outputs)
pretraining_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.experimental.AdamW(PRETRAINING_LEARNING_RATE),
    weighted_metrics=["sparse_categorical_accuracy"],
    jit_compile=True,
)

# Modeli wiki metin veri kümemizde önceden eğitin.
pretraining_model.fit(
    pretrain_ds,
    validation_data=pretrain_val_ds,
    epochs=PRETRAINING_EPOCHS,
)

# SDaha fazla ince ayar için bu temel modeli kaydedin.
encoder_model.save("encoder_model")

## Fine-tuning

Ön eğitimden sonra artık modelimizin ince ayarını "SST-2" veri kümesinde yapabiliriz. Bağlam içindeki kelimeleri tahmin etmek için oluşturduğumuz kodlayıcının yeteneğinden yararlanın.


### Verileri sınıflandırma için ön işleme



In [ ]:

def preprocess(sentences, labels):
    return tokenizer(sentences), labels


# Önceden işlenmiş partileri CPU'muzda hareket halindeyken önceden hesaplamak için prefetch() kullanıyoruz.
finetune_ds = sst_train_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
finetune_val_ds = sst_val_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)


print(finetune_val_ds.take(1).get_single_element())

### Fine-tune Transformer


In [ ]:
# İnce ayarı sıfırdan yeniden başlatabilmemiz için kodlayıcı modelini diskten yeniden yükleyin.
encoder_model = keras.models.load_model("encoder_model", compile=False)

# Girdi olarak tokenleştirilmiş girişi alın.
inputs = keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32)

# Tokenleri encoder edin ve pooling'leyin.
encoded_tokens = encoder_model(inputs)
pooled_tokens = keras.layers.GlobalAveragePooling1D()(encoded_tokens)

# Bir çıktı etiketi tahmin edin.
outputs = keras.layers.Dense(1, activation="sigmoid")(pooled_tokens)

# Hassas ayar modelimizi tanımlayın ve derleyin.
finetuning_model = keras.Model(inputs, outputs)
finetuning_model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.experimental.AdamW(FINETUNING_LEARNING_RATE),
    metrics=["accuracy"],
)

# Modelde SST-2 görevi için ince ayar yapın.
finetuning_model.fit(
    finetune_ds,
    validation_data=finetune_val_ds,
    epochs=FINETUNING_EPOCHS,
)

### Test için bir model kaydedelim.


In [ ]:
inputs = keras.Input(shape=(), dtype=tf.string)
tokens = tokenizer(inputs)
outputs = finetuning_model(tokens)
final_model = keras.Model(inputs, outputs)
final_model.save("final_model")

restored_model = keras.models.load_model("final_model", compile=False)
inference_data = tf.constant(["Terrible, no good, trash.", "So great; I loved it!"])
print(restored_model(inference_data))

KerasNLP'nin temel hedeflerinden biri, NLP model oluşturmaya modüler bir yaklaşım sağlamaktır. Burada bir Transformer oluşturmaya yönelik bir yaklaşım gösterdik, ancak KerasNLP, metin ön işleme ve model oluşturma için sürekli büyüyen bir bileşen dizisini destekler. Doğal dil problemlerinize çözümler üzerinde deneyler yapmayı kolaylaştıracağını umuyoruz.